In [1]:
from hardwareModel import HardwareModel
import networkx as nx
import cvxpy as cp
import numpy as np

In [2]:
hw = HardwareModel(cfg="aladdin_const_with_mem")

netlist: [('Add0', {'type': 'pe', 'function': 'Add', 'in_use': 0, 'idx': 0}), ('Regs0', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 0}), ('Regs1', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 1}), ('Regs2', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 2}), ('Mult0', {'type': 'pe', 'function': 'Mult', 'in_use': 0, 'idx': 0}), ('Eq0', {'type': 'pe', 'function': 'Eq', 'in_use': 0, 'idx': 0}), ('Buf0', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 0, 'size': 22}), ('Buf1', {'type': 'mem', 'function': 'Buf', 'in_use': 0, 'idx': 1, 'size': 22}), ('Mem0', {'type': 'mem', 'function': 'MainMem', 'in_use': 0, 'idx': 0, 'size': 1024}), ('Regs3', {'type': 'memory', 'function': 'Regs', 'in_use': 0, 'size': 1, 'idx': 3})]


In [3]:
print(hw.netlist)
print(hw.area)

DiGraph with 10 nodes and 38 edges
{'Add': 379348800.0, 'Sub': 379348800.0, 'And': 50369960.0, 'BitAnd': 50369960.0, 'BitOr': 50369960.0, 'BitXor': 50369960.0, 'Eq': 379348800.0, 'FloorDiv': 8340007000.0, 'Gt': 379348800.0, 'GtE': 379348800.0, 'Invert': 50369960.0, 'IsNot': 50369960.0, 'LShift': 24964610.0, 'Lt': 379348800.0, 'LtE': 379348800.0, 'Mod': 379348800.0, 'Mult': 8340007000.0, 'Not': 50369960.0, 'NotEq': 379348800.0, 'Or': 50369960.0, 'RShift': 24964610.0, 'Regs': 5981433.0, 'UAdd': 379348800.0, 'USub': 379348800.0, 'Buf': 0.0, 'MainMem': 5981433.0}


In [4]:
hw.area['Add']

379348800.0

In [5]:
digraph = hw.netlist
nodes = digraph.nodes
edges = digraph.edges
#print("Nodes:", list(nodes))
print("Edges:", list(edges))

Edges: [('Add0', 'Regs0'), ('Add0', 'Regs1'), ('Add0', 'Regs2'), ('Add0', 'Mult0'), ('Add0', 'Eq0'), ('Regs0', 'Regs1'), ('Regs0', 'Add0'), ('Regs0', 'Mult0'), ('Regs0', 'Eq0'), ('Regs0', 'Buf1'), ('Regs1', 'Regs0'), ('Regs1', 'Add0'), ('Regs1', 'Mult0'), ('Regs1', 'Eq0'), ('Regs1', 'Buf1'), ('Regs2', 'Regs3'), ('Regs2', 'Add0'), ('Regs2', 'Mult0'), ('Regs2', 'Eq0'), ('Regs2', 'Buf1'), ('Mult0', 'Add0'), ('Mult0', 'Regs0'), ('Mult0', 'Regs1'), ('Mult0', 'Regs2'), ('Mult0', 'Eq0'), ('Eq0', 'Add0'), ('Eq0', 'Mult0'), ('Eq0', 'Regs0'), ('Eq0', 'Regs1'), ('Eq0', 'Regs2'), ('Buf1', 'Regs3'), ('Buf1', 'Regs0'), ('Buf1', 'Mem0'), ('Buf1', 'Regs1'), ('Buf1', 'Regs2'), ('Mem0', 'Buf1'), ('Regs3', 'Regs2'), ('Regs3', 'Buf1')]


In [6]:
node_to_index = {node: i for i, node in enumerate(nodes)}
n = len(nodes) 
E = [(node_to_index[src], node_to_index[dst]) for src, dst in edges]

# Some fixed lij values for each edge (will slowly decrease the value until it doesn't work)
l = {edge: 10 for edge in E}

# Random weights for example, you may replace or adjust these
#w = np.random.randn(n) #fix this

w = np.asarray([[1] for _ in node_to_index])
W = cp.diag(w)
print(f'W matrix: {W}')
#w = np.abs(np.random.randn(n))
#w = list(hw.area.values())

p = cp.Variable( (n, 2) )


#objective = cp.Maximize(cp.quad_form(p, np.diag(w)))

# sum(diag(W(pp^T)) (Implement first)

#objective = cp.Maximize( w[0] * (cp.norm(p[0]) ** 2) ) 
# wDiag = cp.diag(w)
# product = p @ p.T
# objective = cp.Maximize(cp.sum(cp.multiply(wDiag, product)))

# pSquared = np.array([cp.norm(p[i]) for i in range(n)]) ** 2
# mult = cp.multiply(w, pSquared)
# objective.cp.Maximize(sum(mult))

# pPT = p @ p.T
# mult = w.T @ pPT
# diagonals = cp.diag(w.T @ p @ p.T)

#objective = cp.Minimize(-cp.sum(cp.diag(p @ p.T)))
objective = cp.Minimize(-sum([w[i] * (cp.norm(p[i]) ** 2) for i in range(len(w))]))

constraints = [cp.sum(cp.multiply(w, p)) == 0]  # Sum of w_i * p_i = 0

# Adding |p_i - p_j| <= l_ij constraints for each (i, j) in E
for (i, j) in E:
    constraints += [cp.abs(p[i] - p[j]) <= l[(i, j)]]

problem = cp.Problem(objective, constraints)
problem.solve()

W matrix: diag_vec(reshape([[1.00]
 [1.00]
 ...
 [1.00]
 [1.00]], (10,), F), 0)


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to minimize a function that is concave.

In [10]:
def compute_Kij(i, j, n):
    # Create the unit vectors e_i and e_j
    e_i = np.zeros(n)
    e_j = np.zeros(n)
    e_i[i] = 1
    e_j[j] = 1
    
    # Compute the difference
    diff = e_i - e_j
    
    # Compute the outer product of the difference with itself
    Kij = np.outer(diff, diff)
    
    return Kij

In [11]:
compute_Kij(0, 5, 6)

array([[ 1.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0., -0.],
       [ 0.,  0.,  0.,  0.,  0., -0.],
       [-1., -0., -0., -0., -0.,  1.]])

In [43]:
node_to_index = {node: i for i, node in enumerate(nodes)}
# E = []

# for src, dst in edges:
#     E.append((node_to_index[src], node_to_index[dst]))
    

E = [(node_to_index[src], node_to_index[dst]) for src, dst in edges]
n = len(nodes)
w = np.asarray([[0] for _ in node_to_index])

for node in nodes:
    index = node_to_index[node]
    updatedName = node[:len(node) - 1]
    updatedName = 'MainMem' if updatedName == 'Mem' else updatedName
    w[index, 0] = hw.area[updatedName]

print(w)

print(nodes)
print(hw.area)

#w = np.asarray([[1] for _ in node_to_index])
diag_w = cp.diag(w)
wwT = w @ w.T
Ti = 4.7
di = 1
alpha = 1
Q = cp.Variable( (n, n), PSD=True)
t = cp.Variable(n)

objective = cp.Maximize(cp.sum(cp.multiply(diag_w, Q))) 

constraints = [cp.sum(cp.multiply(wwT, Q)) == 0, Q >= 0] 

for i in range(n):
    constraints += [t[i] <= Ti]
    
for pair in E:
    if pair[0] == i:
        constraints += [t[i] + di + alpha * cp.sum(cp.multiply(compute_Kij(i, pair[1], n), Q)) <= t[pair[1]]]

prob = cp.Problem(objective, constraints)
prob.solve(solver='SCS', verbose=False)

print("The optimal value is", prob.value)
print("A solution t is", t.value)
print("A solution Q is", Q.value)

[[ 379348800]
 [   5981433]
 [   5981433]
 [   5981433]
 [8340007000]
 [ 379348800]
 [         0]
 [         0]
 [   5981433]
 [   5981433]]
['Add0', 'Regs0', 'Regs1', 'Regs2', 'Mult0', 'Eq0', 'Buf0', 'Buf1', 'Mem0', 'Regs3']
{'Add': 379348800.0, 'Sub': 379348800.0, 'And': 50369960.0, 'BitAnd': 50369960.0, 'BitOr': 50369960.0, 'BitXor': 50369960.0, 'Eq': 379348800.0, 'FloorDiv': 8340007000.0, 'Gt': 379348800.0, 'GtE': 379348800.0, 'Invert': 50369960.0, 'IsNot': 50369960.0, 'LShift': 24964610.0, 'Lt': 379348800.0, 'LtE': 379348800.0, 'Mod': 379348800.0, 'Mult': 8340007000.0, 'Not': 50369960.0, 'NotEq': 379348800.0, 'Or': 50369960.0, 'RShift': 24964610.0, 'Regs': 5981433.0, 'UAdd': 379348800.0, 'USub': 379348800.0, 'Buf': 0.0, 'MainMem': 5981433.0}
The optimal value is inf
A solution t is None
A solution Q is None
